## Covid vaccination centres recommendations based on distance.



### Information

The COVID-19 vaccination programme seeks to protect Singaporeans against COVID-19, as well as to protect businesses and jobs through the progressive re-opening of Singapore. Vaccination is free to all Singaporeans and long-term residents in Singapore.

Visit https://www.vaccine.gov.sg/ today to book your appointment today.

Code written by Topaz Tan, Michael Ng and Daren Neo

### Explanation

This notebook comprises two parts. Part 1 explains our code in detail. Part 2 prepares our functions. Part 3 provides an example of how our application will run. It will have functions that users can call to use our application.

### For users
Please run all the cells in Parts 1 and 2 before continuing to Part 3.

-----
## Part 1

### Find your nearest vaccination centre
Welcome! Here, you can find out the nearest vaccination centre near you.

First, install all the necessary libraries.

In [ ]:
!pip install geopy
!pip install haversine
!pip install folium
import pandas as pd
import folium # map rendering library
import haversine as hs #Haversine function
from geopy.geocoders import Nominatim

print('Libraries imported.')

Next, we will load the data we need. 

Data retrieved from https://www.vaccine.gov.sg/locations-vcs

In [ ]:
cc_list=['Radin Mas CC','Kolam Ayer CC','Buona Vista CC','Potong Pasir CC','Raffles Convention Centre','Tanjong Pagar CC','Jalan Besar CC','Bishan CC','Queenstown CC','Toa Payoh West CC','Marine Parade CC','Bukit Timah CC','Geylang Serai CC','Tampines East CC','Changi Airport T4','Bedok CC','Pasir Ris Elias CC','Arena@OTH Tampines','Marsiling CC','Woodlands CC','Woodlands Galaxy CC','Canberra CC','Nee Soon East CC','Kebun Baru CC','Punggol 21 CC','Serangoon CC','Sengkang CC','Hougang CC','Teck Ghee CC','Hong Kah North CC','Yew Tee CC','Former Hong Kah Secondary School','Taman Jurong CC','Yuhua CC','Nanyang CC','Clementi CC','Senja Cashew CC','Chua Chu Kang CC'] 

cc_regions = ['Central','Central','Central','Central','Central','Central','Central','Central','Central','Central','Central','Central','Central','East','East','East','East','East','North','North','North','North','North','Northeast','Northeast','Northeast','Northeast','Northeast','Northeast','West','West','West','West','West','West','West','West','West']

cc_latitudes = [1.276,1.324,1.309,1.333,1.293,1.276,1.308,1.35,1.299,1.335,1.305,1.341,1.316,1.353,1.338,1.324,1.378,1.352,1.441,1.44,1.439,1.445,1.431,1.373,1.393,1.37,1.3935,1.365,1.363,1.359,1.395,1.352,1.335,1.34,1.342,1.319,1.381,1.381]

cc_longitudes = [103.82,103.87,103.792,103.867,103.854,103.842,103.862,103.85,103.801,103.845,103.91,103.772,103.897,103.955,103.984,103.936,103.943,103.94,103.774,103.788,103.802,103.82,103.8385,103.8377,103.914,103.874,103.894,103.892,103.853,103.749,103.7447,103.727,103.722,103.737,103.692,103.768,103.7646,103.752]

cc_contacts = [62735294,62935572,67785163,62801182,63397777,62219898,62986110,62594720,64741681,63530577,63467703,
64662912,67472919,67863227,65956868,64425317,65831220,63403636,62696768,63689938,63662218,67556369,
62570446,64577379,63120508,62858833,63125400,62828887,64567123,65673130,67693672,65679655,62651711,
65604490,67910395,67762517,62194561,67691694]

cc_addresses = ['51 Telok Blangah Crescent 098917','1 Geylang Bahru Lane 339631','36 Holland Drive 270036','6 Potong Pasir Avenue 2 358361',
                '80 Bras Basah Road 189560','101 Cantonment Road 089774','69 Jellicoe Road 208737','51 Bishan Street 13 579799',
                '365 Commonwealth Avenue 149732','200 Lorong 2 Toa Payoh 319642','278 Marine Parade Road 449282','20 Toh Yi Drive 596569',
                '1 Engku Aman Turn 408528','10 Tampines Street 23 529341','10 Airport Boulevard 819665','850 New Upper Changi Road 467352',
                '93 Pasir Ris Drive 3 519498','1 Tampines Walk 528523','100 Admiralty Road 739980','1 Woodlands Street 81 738526',
                '31 Woodlands Ave 6 738991','2 Sembawang Crescent 757632','1 Yishun Avenue 9 768893','216 Ang Mo Kio Avenue 4 569897',
                '80 Punggol Field 828815','10 Serangoon North Avenue 2 555877','2 Sengkang Square 525025','35 Hougang Avenue 3 538840',
                '861 Ang Mo Kio Avenue 10 569734','30 Bukit Batok Street 31 659440','20 Choa Chu Kang Street 52 689286','931 Jurong West Street 42 649370',
                '1 Yung Sheng Road 618495','90 Boon Lay Way 609958','60 Jurong West Street 91 649040','220 Clementi Avenue 4 129880',
                '101 Bukit Panjang Road 679910','35 Teck Whye Avenue 688892']

cc_type = ['Moderna','Moderna','Moderna','Moderna','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer','Moderna','Pfizer','Pfizer','Pfizer','Pfizer','Moderna','Moderna','Pfizer','Pfizer','Pfizer','Moderna','Moderna','Pfizer','Pfizer','Pfizer','Pfizer','Moderna','Moderna','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer','Pfizer']    

In [3]:
coordinates_tuples = list()
for i in range(38):
  coordinates_tuples.append((cc_latitudes[i], cc_longitudes[i]))
cc_dict = dict(zip(cc_list,coordinates_tuples))

In [ ]:
info = pd.DataFrame(
	{'Name': cc_list,
 	'Region': cc_regions,
 	'Contact No.':cc_contacts,
 	'Latitude': cc_latitudes,
 	'Longitude': cc_longitudes,
   'Address': cc_addresses,
  'Vaccine': cc_type
  })
info #displays the data frame

Now, we shall find the nearest vaccination centers!

### Distance function to determine recommendation based on distance

<div class="alert alert-block alert-warning">
<b>
Please enter a valid address below (e.g. Junction 8, Jurong Point, 313 Orchard Road, Sentosa, Sengkang)
</b>
</div>

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
your_Add = input("Enter your address (block, street,country): ")
user = locator.geocode(your_Add)
if user is None:
  print ("Please enter a valid address. Try again")

The Geocode function returns 2 values, the longitude and latitude.

In [ ]:
print(user)
print(type(user))
print(user.latitude, user.longitude)

The variable 'user' contains all the information we need. We shall use Geopy's built-in methods 'latitude' and 'longitude'.

Now, we can calculate the distance (in km) from the location selected by the user to each of the vaccination centres.

In [ ]:
loc1 = (user.latitude, user.longitude)
distance_dict = {}
for loc2 in cc_dict:
 loc2_coor = cc_dict[loc2] #access the loc2 key, get the value of that key
 distance_km = hs.haversine(loc1, loc2_coor)
 distance_dict[str(loc2)]= distance_km
print(distance_dict)

We sort the dictionary *distance_dict* based from nearest to farthest.

In [ ]:
distance_sorted = sorted(distance_dict.items(), key=lambda x: x[1])
print(distance_sorted)

### Map Display


Now, we can display this information in an interactive way as well!

We shall use the python library 'Folium', which provides an intuitive design.

First we shall initialise our map.

In [11]:
#Center the map on user's location
map_singapore = folium.Map(width=1600,height=900,
                           location=[user.latitude,user.longitude], 
                           zoom_start=13.5)

Add our markers for our vaccination centres.

In [12]:
for index,row in info.iterrows():
    folium.Marker(
        [row.loc['Latitude'],row.loc['Longitude']],
        popup=row.loc['Name'] + ' Phone: ' + str(row.loc['Contact No.']) +'       '+row.loc['Address'] +' Type: '+ row.loc['Vaccine'],
        tooltip=row.loc['Name']).add_to(map_singapore)

Finally, let's look at our map.

Please run the code, and type how many locations to display.

The red maker shows the user's desired location. Hover your cursor over the markers and click for more information.


In [ ]:
#Show the user's location
folium.Marker(
    location=[user.latitude, user.longitude], 
    tooltip="Your Location",
    popup="Your Location",
    icon=folium.Icon(color="red", icon="street-view", prefix="fa")
    ).add_to(map_singapore)


print("The recommended Covid Vaccincation Centres are: ")
count = 0
usercount = int(input("How many locations would you like us to recommend? "))
for (a,b) in distance_sorted:
 print (a,round(b,3), "km")
 count += 1
 if count < usercount:
  continue 
 else: 
   break

map_singapore

## Part 2: Preparing Our Functions

We will define four functions for the user to use.

In [ ]:
def locate():
    locator = Nominatim(user_agent="myGeocoder")
    your_Add = input("Enter your address (block, street,country): ")
    user = locator.geocode(your_Add)
    if user is None:
      print ("Please enter a valid address. Try again")
    loc1 = (user.latitude, user.longitude)
    distance_dict = {}
    for loc2 in cc_dict:
        loc2_coor = cc_dict[loc2] #access the loc2 key, get the value of that key
        distance_km = hs.haversine(loc1, loc2_coor)
        distance_dict[str(loc2)]= distance_km
    distance_sorted = sorted(distance_dict.items(), key=lambda x: x[1])
    map_singapore = folium.Map(width=1600,height=900,location=[user.latitude,user.longitude], zoom_start=13.5)
    for index,row in info.iterrows():
        folium.Marker([row.loc['Latitude'],row.loc['Longitude']],popup=row.loc['Name'] + ' Phone: ' + str(row.loc['Contact No.']) +'       '+row.loc['Address'] +' Type: '+ row.loc['Vaccine'], tooltip=row.loc['Name']).add_to(map_singapore)
    folium.Marker(location=[user.latitude, user.longitude], tooltip="Your Location",popup="Your Location",icon=folium.Icon(color="red", icon="street-view", prefix="fa")).add_to(map_singapore)


    print("The recommended Covid Vaccincation Centres are: ")
    count = 0
    usercount = int(input("How many locations would you like us to recommend? "))
    for (a,b) in distance_sorted:
     print (a,round(b,3), "km")
     count += 1
     if count < usercount:
      continue 
     else: 
       break

    display(map_singapore)

In [ ]:
def search():
  print('Enter the CC name: ')
  selected1 = input()
  return info.loc[info['Name'] == selected1]

In [ ]:
def filter_by_vaccine():
  selected2 = input()
  if selected2=='Pfizer' or selected2=='Moderna':
    print(info.loc[info['Vaccine'] == selected2] )
  else:
    print('Invalid input. Please try again.')

In [ ]:
def filter_by_region():
  selected3 = input()
  if selected3=='North' or selected3=='Northeast' or selected3=='East' or selected3=='Central' or selected3=='West':
    print(info.loc[info['Region'] == selected3] )
  else:
    print('Invalid input. Please try again.')

## Part 3: User interface

<div class="alert alert-block alert-info">

<b>Hello! This application will help you find your desired vaccination center.</b>
    
   
1. To find the vaccination centers near you, type 'locate()' in the cell below.
    

2. To find out more details about the nearest vaccination centres, type 'search()' in the cell below.
    

3. To find out which vaccination centers offers either Pfizer/Moderna, type 'filter_by_vaccine()' in the cell below.
    

4. To find out which vaccination centers are in your region, type 'filter_by_region()' in the cell below.
    
</div>


### 1. locate()
This function condenses the code explained earlier into a user-friendly function.

<div class="alert alert-block alert-info">

1. Ensure that you have run the code in Parts 1 and 2.
    
2. Type locate()
    
3. Press Enter

4. Type your desired location
    
5. Type how many locations you would like to show
    
6. (Optional) Run the cell again to select a different address.
</div>

In [37]:
locate()

## Search function

Lastly, we have three search functions to find out more information about the nearest vaccination centres.

### 2. search()

Find information about any vaccination centre.


<div class="alert alert-block alert-info">

1. Type search()
    
2. Press Enter

3. Type the name of the Vaccination Centre (eg. Bishan CC, Serangoon CC)
</div>

In [ ]:
search()

### 3. filter_by_vaccine

This function help you find the vaccination center offering the vaccine you want, either Pfizer or Moderna.

<div class="alert alert-block alert-info">

1. Type 'filter_by_vaccine()'
    
2. Press Enter

3. Type either 'Pfizer' or 'Moderna'
</div>

In [ ]:
filter_by_vaccine()

### 4. filter_by_region

A way for you to easily find the vaccination center offering the vaccine you want.

<div class="alert alert-block alert-info">

1. Type 'filter_by_region()'
    
2. Press Enter

3. Type either 'North', 'Northeast', 'East', 'Central', or 'West'.
</div>

In [ ]:
filter_by_region()

Thank you for using our service. We hope our application has served you well.

To book your vaccination, please visit https://www.vaccine.gov.sg/